<a href="https://colab.research.google.com/github/shannonfernandes25/bioinformatics-BPRI-Bioinformatics/blob/main/negative_embedding_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Transformer - encoding - converting our alphabetical data into numeric form - HuggingFace

torch - PyTorch

tqdm - progress bars

In [1]:
!pip install transformers tqdm biopython torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 25.8 MB/s eta 0:00:00


In [2]:
import torch
from tqdm import tqdm
import pandas as pd
from transformers import AutoTokenizer, AutoModel

In [3]:
# Load your data
df = pd.read_excel("/content/shannon_negative_data.xlsx")

# Ensure 'Sequence' column exists and converts to list
sequences = df["Sequence"].tolist()

Potein embedding models-
1. protbert

   a. protbert_03

   b. protbert_05

2. ESM

   a. ESM_t6
   
   b. ESM_t12

max_length = 1024

truncation = True  (cut long sequences as per token length)


In [4]:
# generate mean embeddings
def generate_embeddings(model_name, sequences, device):
    tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False)
    model = AutoModel.from_pretrained(model_name).to(device)
    model.eval()

    embeddings = []
    with torch.no_grad():
        for seq in tqdm(sequences):
            # Tokenize and convert to tensor
            tokens = tokenizer(seq, return_tensors='pt', truncation=True, max_length=1024)
            tokens = {k: v.to(device) for k, v in tokens.items()}

            outputs = model(**tokens)
            # Take mean across all residues
            emb = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
            embeddings.append(emb)
    return embeddings

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

All the encoding model will generate embeddings per residue but we want the embeddings per protein so to get that, we are calculating mean of the embeddings per residue.

device -  parameters to GPU (cuda) or CPU

model.eval() - put the model in evaluation mode (disable other training mode)

 torch.no_grad - it is a context manager which will disable gradient computation so that it will not take extra memory

In [5]:
# Generate embeddings for each model
models = {
    "ProtBERT03": "Rostlab/prot_bert_bfd",
    "ProtBERT05": "Rostlab/prot_bert",
    "ESM_t6": "facebook/esm2_t6_8M_UR50D",
    "ESM_t12": "facebook/esm2_t12_35M_UR50D"
}

# Generate and store
all_embeddings = {}
for label, model_name in models.items():
    print(f"\n🔹 Generating {label} embeddings...")
    all_embeddings[label] = generate_embeddings(model_name, sequences, device)

# Combine all embeddings into one DataFrame
for label in all_embeddings:
    emb_df = pd.DataFrame(all_embeddings[label])
    emb_df.columns = [f"{label}_{i}" for i in range(emb_df.shape[1])]
    df = pd.concat([df, emb_df], axis=1)

# Save to CSV
output_path = "protein_embeddings.csv"
df.to_csv(output_path, index=False)
print(f"\n✅ Embeddings saved to: {output_path}")


🔹 Generating ProtBERT03 embeddings...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/778 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/136M [00:00<?, ?B/s]

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t12_35M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 510/510 [00:53<00:00,  9.47it/s]



✅ Embeddings saved to: protein_embeddings.csv
